In [19]:
### 融资融券对股价的影响
import h5py
import pandas as pd
from datetime import datetime

In [37]:
# 特征加工
with h5py.File('D:/data_file/jqdata.hdf5', 'r') as files:
    for key in files.keys():
        print(dir(files[key]))
        print(files[key].keys)
        assert 1==2

['_MutableMapping__marker', '__abstractmethods__', '__bool__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_d', '_e', '_gcpl_crt_order', '_id', '_lapl', '_lcpl', 'attrs', 'clear', 'copy', 'create_dataset', 'create_dataset_like', 'create_group', 'create_virtual_dataset', 'file', 'get', 'id', 'items', 'keys', 'move', 'name', 'parent', 'pop', 'popitem', 'ref', 'regionref', 'require_dataset', 'require_group', 'setdefault', 'update', 'values', 'visit', 'visititems']


RuntimeError: Can't determine (bad symbol table node signature)

In [11]:
trade_days_str[trade_days_str.index(target_day_str)-20:trade_days_str.index(target_day_str)]

['2021-01-15',
 '2021-01-18',
 '2021-01-19',
 '2021-01-20',
 '2021-01-21',
 '2021-01-22',
 '2021-01-25',
 '2021-01-26',
 '2021-01-27',
 '2021-01-28',
 '2021-01-29',
 '2021-02-01',
 '2021-02-02',
 '2021-02-03',
 '2021-02-04',
 '2021-02-05',
 '2021-02-08',
 '2021-02-09',
 '2021-02-10',
 '2021-02-18']